<div class="alert alert-block alert-success">
    <h1>DataLab HvA - serie Machine Learning Python - Tech report 5</h1>
</div>

<div class="alert alert-block alert-warning">
    <h2>Logistische Regressie</h2>
</div>

### 0. Doel van deze les
Vanwege de serie lessen waarbinnen deze les valt is het hierin de insteek is om het gemaakte model te gaan vergelijken met andere andere Machine Learning modellen en daarbij een kwaliteitscriterium te hanteren waarmee *alle* modellen onderling vergelijkbaar zijn. Dit is de reden dat we werken met een test- en een trainset van onze data.
**Je leert** 
- hoe je in Python met behulp van het package `sklearn` een Logistisch Regressiemodel voor een binaire classificatietaak kunt maken
- hoe je de geevalueerde modelcoëfficiënten van dit model kunt bekijken
- hoe je dit model kunt gebruiken om voorspellingen te genereren op nieuwe data

**Je leert niet:**
- algemene theoretisch wiskundige achtergronden over Logistische Regressie, hiervoor verwijzen we naar een les statistiek in je eigen opleiding of één van de vele online tutorials hierover.
- bij meervoudige logistisceh regressie: strategiën met betrekking tot het selecteren van de in het model te gebruiken variabelen (*feature selection*). 
- hoe je met het package `seaborn` eerste visuele verkenningen van de (variatie in de) data kan uitvoeren, zie hiervoor *Tech report 1: Introductie Machine Learning* uit deze serie.
- hoe je data importeert en inlaadt in de python sessie, zie hiervoor echter wel [deze instructie](https://www.youtube.com/watch?v=1oBVx7pyuXo&list=PLwkTCAI_gJjW10xaAJpdqeQx7EQgFolfG&index=10) 

**Machine Learning framework**
- Om een consistente leerlijn in de serie Machine Learning aan te houden kiezen we telkens voor een opzet waarin een inititeel model bepaald wordt o.b.v. een *deel* van de beschikbare data: de *train data*. Om het model te kunnen vergelijken met ándere modeltypen (waarbij andere algoritmen worden gebruikt) stellen we de voorspellende prestaties vast a.h.v. de nog niet gebruikte data: de *test data*. 

### 1. Laden van de benodigde packages
Packages die je zelf nog niet hebt geinstalleerd dien je vooraf in Python te installeren. 

In [1]:
# Voor de dataset
import seaborn as sns

# Voor het bewerken en inspecteren van de data
import pandas as pd

# Voor het afsplitsen van een train- en test dataset
from sklearn.model_selection import train_test_split

# Voor het modelleren
from sklearn.linear_model import LogisticRegression

### 2. Laden en prepareren van de data

We gebruiken als voorbeeld een dataset `'tips'`: "*One waiter recorded information about each tip he received over a period of a few months working in one restaurant. He collected several variables*". In totaal beschikken we over 244 waarnemingen. Lees eventueel meer over deze set [hier](https://rdrr.io/cran/reshape2/man/tips.html).   

#### Laden en weergave data

In [2]:
weg = sns.load_dataset('tips')
weg['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [3]:
# Laden van een standaard dataset uit package seaborn 
data = sns.load_dataset('tips')
print(data.head())
print(data.shape)
print(data.dtypes)

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4
(244, 7)
total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
dtype: object


#### Dummy encoding categoriale variabelen
We zien dat we te maken hebben met een aantal categoriale variabelen, het logistische regressiemodel kan hier niet mee overweg (zie ook de korte toelichting op het modeltype hieronder). We maken hiervoor *dummie variabelen* aan. Om ervoor te zorgen dat deze onderling niet volledig gecorreleerd zijn zal er één dummy minder worden aangemaakt dan er mogelijke waarden zijn (`drop_first=True`) :

In [4]:
data = pd.get_dummies(data=data, drop_first=True)
print(data.head())
print(data.shape)
print(data.dtypes)

   total_bill   tip  size  sex_Female  smoker_No  day_Fri  day_Sat  day_Sun  \
0       16.99  1.01     2           1          1        0        0        1   
1       10.34  1.66     3           0          1        0        0        1   
2       21.01  3.50     3           0          1        0        0        1   
3       23.68  3.31     2           0          1        0        0        1   
4       24.59  3.61     4           1          1        0        0        1   

   time_Dinner  
0            1  
1            1  
2            1  
3            1  
4            1  
(244, 9)
total_bill     float64
tip            float64
size             int64
sex_Female       uint8
smoker_No        uint8
day_Fri          uint8
day_Sat          uint8
day_Sun          uint8
time_Dinner      uint8
dtype: object


Voor de taak van aanmaken van Dummies of One-Hot-Encodig kan ook gebruik worden gemaakt van de scikit learn functies  `OneHotEncoder, LabelEncoder`. Echter ontstaan daarna objecte van het type `'numpy.ndarray'` (matrices) waarin bijvoorbeeld géén namen van de variabelen meer te vinden zijn. We geven hier de voorkeur aan de functie `get_dummies()` uit de `pandas` library waarin een panda's data frame wordt aangemaakt.

#### Split variabelen X en y
We gaan in dit voorbeeld de oorspronkelijke variabele `time` voorspellen, de oorspronkelijke variabele : indien de waarneming betrekking heeft op een maaltijd in de avond geldt na dummy encoding: `time_Dinner = 1` (`Diner` is dus de positieve klasse) en wanneer dit in de middag is: `time_Dinner = 0` (`Lunch` is de negatieve klasse). Zie meer toelichting bij *sectie 3: Voorbeschouwing*. 

In [5]:
#independent variables / explanatory variables
X = data.drop('time_Dinner', axis=1)

#dependent variable / response / target variable.
y = data['time_Dinner']

#### Afsplitsen test data en train data
Voor meer details over deze stap, zie *Tech report 2: Datapreparatie* uit deze serie.

In [6]:
# Split in training en test data 
# random_state kiezen zorgt voor reproduceerbaarheid van dit resultaat - gebruik dat ! Een andere waarde is uiteraard ook prima
# we kiezen voor een verhouden train - test van 70% - 30%. Andere keuzen kunnen ook!
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.30,random_state=999)
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

(170, 8)
(74, 8)
(170,)
(74,)


In [7]:
X.head()

,total_bill,tip,size,sex_Female,smoker_No,day_Fri,day_Sat,day_Sun
0,16.99,1.01,2,1,1,0,0,1
1,10.34,1.66,3,0,1,0,0,1
2,21.01,3.50,3,0,1,0,0,1
3,23.68,3.31,2,0,1,0,0,1
4,24.59,3.61,4,1,1,0,0,1


### 3. Voorbeschouwing
Zomaar modelleren doe je niet, eerst moeten we vaststellen wat we willen bereiken of welke vraag we zouden willen beantwoorden. Voor deze les zal dat worden:
- Taak: stel een logistisch regressie model op om het tijdstip van de maaltijd (*Diner* of *Lunch*) te voorspellen op basis van de overige variabelen. Dit is een *Binaire Classificatietaak*. 

Vanuit de taak zullen we dus stellen:
- De *onafhankelijke* variabelen ($x_i$) zijn de variabelen `total_bill`, `tip`, `size`, `sex_Female`, `smoker_No`, `day_Fri`, `day_Sat`, `day_Sun`,  
- De *afhankelijke* variabele ($y$) is de variabele `time_Dinner`

We zijn, met bovengenoemde definitie van $x_i$ en $y$ op zoek naar het model:
$$ P(y = 1) = \frac{1}{1+ e^{-(\alpha + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_k x_k +\epsilon)}}$$

Met in ons geval $k = 8$

Dat wil zeggen: we willen de *modelparameters* $\alpha$ en $\beta_i$ vaststellen. De term $\epsilon$ drukt de *fout* uit die elke voorspelling onvermijdelijk zal hebben. De algoritmen in `sklearn` die ons dit model gaan geven zorgen ervoor dat deze *fout* (eigelijk: het kwadraat ervan) voor de voorspellingen op de gebruikte traindata gemiddeld genomen zo klein mogelijk zal zijn. 

### 4. Model schatten
#### lees documentatie bij package
Voordat we een model gaan schatten is het van belang dat je de documentatie van het algoritme dat we uit het package gaan gebruiken lezen. In dit geval is dat dus voor `LinearRegression()`, die lees je bijvoorbeeld online [hier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Alternatief is om de `help()` functie te gebruiken, zie hieronder. 

In [8]:
# We gaan de methode LinearRegression() uit het sklearn package gebruiken, inspecteer hiervoor de documentatie. Dat kan 
help(LogisticRegression())

Help on LogisticRegression in module sklearn.linear_model._logistic object:

class LogisticRegression(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
 |  LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
 |  
 |  Logistic Regression (aka logit, MaxEnt) classifier.
 |  
 |  In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
 |  scheme if the 'multi_class' option is set to 'ovr', and uses the
 |  cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
 |  (Currently the 'multinomial' option is supported only by the 'lbfgs',
 |  'sag', 'saga' and 'newton-cg' solvers.)
 |  
 |  This class implements regularized logistic regression using the
 |  'liblinear' library, 'newton-cg', '

Zie bijvoorbeeld de parameter: `fit_intercept=True` (default waarde), als je die `=False` kiest zul je een $\alpha = 0$ afdwingen in het model. Ook lees je dat er een *method*  `fit()` voor deze *object class* gedefinieerd is om het model te *fitten*.

#### model fitten
We gebruiken dus de train data om een model te fitten. Dit is, zoals je ziet, een eenvoudig regeltje code. We geven het model een naam (`model_logreg`) zodat we het later weer opnieuw kunnen aanroepen / gebruiken.

In [9]:
# Met model fitten default van package
model_logreg = LogisticRegression().fit(train_X, train_y)

### 5. Model coëfficiënten en metrics van model zelf
We stellen vast welke modelcoëfficiënten $\alpha$ en $\beta_i$ het model voor ons heeft geschat. 

In [10]:
# Model mét constante
# coefficiënt alpha
print("alpha =", model_logreg.intercept_)
# coefficient beta
print("[beta_1, beta_2, ..., beta_8] =", model_logreg.coef_)

alpha = [-1.75143748]
[beta_1, beta_2, ..., beta_8] = [[ 0.04124088 -0.08429253  0.02719011 -0.01344589 -0.10134649  1.12419876
   3.90333461  3.78124771]]


### 6. Voorspellingen en perfomance op testset
Hieronder genereren we voorspellingen van y op de testset. Tevens produceren we een modelscore in termen van prestatiematen (performance measures, metrics) op de testset. Voor meer toelichting daarover verwijzen we naar *Tech report 3: performance metrics regressie en classificatie*

In [11]:
predicted_y = model_logreg.predict(test_X)

In [12]:
from sklearn import metrics
# accuracy
print("De accuracy op de testset = ", metrics.accuracy_score(test_y, predicted_y))

# precision
print("De precision op de testset = ", metrics.precision_score(test_y, predicted_y))

# recall
print("De recall op de testset = ", metrics.recall_score(test_y, predicted_y))

# f1-score
print("De f1-score op de testset = ", metrics.f1_score(test_y, predicted_y))

from sklearn.metrics import roc_auc_score
#AUC op testset
auc_test = roc_auc_score(test_y, predicted_y)
print("AUC Testset:", auc_test)

from sklearn.metrics import confusion_matrix
print("Confusion Matrix Testset: \n", confusion_matrix(test_y, predicted_y))

De accuracy op de testset =  0.9459459459459459
De precision op de testset =  1.0
De recall op de testset =  0.9215686274509803
De f1-score op de testset =  0.9591836734693878
AUC Testset: 0.9607843137254901
Confusion Matrix Testset: 
 [[23  0]
 [ 4 47]]
